# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID-19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da Sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD-COVID-19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://covid19.ibge.gov.br/pnad-covid/

Outras fontes:

https://www.gov.br/saude/pt-br/assuntos/noticias/2021/janeiro/entenda-a-ordem-de-vacinacao-contra-a-covid-19-entre-os-grupos-prioritarios

https://www.gov.br/saude/pt-br/assuntos/coronavirus/vacinas/esquemas-vacinais

### Instalando dependências e bibliotecas

In [ ]:
'''!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #instalando java 8
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz #baixando Spark
!tar xf spark-3.1.2-bin-hadoop2.7.tgz #descompactando Spark
!pip install -q findspark'''

### Importando bibliotecas

In [ ]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
import os


os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.1.2-bin-hadoop2.7'

In [ ]:
import findspark

findspark.init()

In [ ]:
#bibliotecas dataviz
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

### Criando sessão Spark

In [ ]:
import findspark


findspark.init()

In [15]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.master('local[*]').appName('Iniciando com Spark').getOrCreate()

In [16]:
spark

### Importando dados

In [17]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import zipfile

#descompactando arquivo zip no drive para caminho especificado
zipfile.ZipFile('/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/2023-10-27_pnad_covid_view.zip', 'r').extractall('/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados') #

In [55]:
path = '/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/2023-11-01_pnad_covid_view.csv' #caminho do arquivo descompactado
df = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [71]:
df.show(5)

2650459 530091


In [ ]:
df.printSchema()

### Análise exploratória

In [ ]:
df_temp = df.createOrReplaceTempView('df_temp') #criando tabela temporária na sessão Spark SQL

#### **<p style='color:gray'> Comportamento dos entrevistados em relação a busca por cuidados médicos</p>**

**Qual foi a quantidade de infectados que buscou atendimento médico?**

In [ ]:
df_testes_positivos_estabelecimento_saude = spark.sql(

'''
    SELECT
        count(resultado_teste) AS qtd_testes_positivos,
        CASE
        WHEN questao_estabelecimento_saude = 'Sim' THEN 'Buscou atendimento'
        WHEN questao_estabelecimento_saude = 'Não' THEN 'Não buscou atendimento'
        WHEN questao_estabelecimento_saude = 'Ignorado' THEN 'Ignorado'
        ELSE 'Não aplicável'
        END AS estabelecimento_saude
    from df_temp
    WHERE resultado_teste = 'Positivo'
    GROUP BY estabelecimento_saude

'''
).toPandas()

fig = px.bar(
    data_frame = df_testes_positivos_estabelecimento_saude,
    x = 'estabelecimento_saude',
    y = 'qtd_testes_positivos',
    color= 'estabelecimento_saude',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'estabelecimento_saude': 'Comportamento'
    }
)

fig.update_layout(
    title='<b>Pessoas que buscaram atendimento</b>',
    xaxis_title='Comportamento',
    yaxis_title='Quantidade',
    showlegend=False,
    width=800,
    height=600
)

fig.show()

#### **<p style='color:gray'> Comportamento dos entrevistados em relação a medidas de isolamento social</p>**

**Qual é a porcetagem de pacientes sintomáticos que efetuou medida de isolamento social?**

In [ ]:
dicionario = {'Sim': 'Sim', 'Não': 'Não', 'Não aplicável': 'Não', 'Ignorado': 'Não'}

df_sintomaticos_permaneceu_casa = spark.sql(
    '''
        SELECT questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos
        FROM df_temp 
        WHERE sintoma_covid = 'Sim'
        GROUP BY questao_permaneceu_casa
    '''
).toPandas()

df_sintomaticos_permaneceu_casa['questao_permaneceu_casa'] = df_sintomaticos_permaneceu_casa['questao_permaneceu_casa'].map(dicionario)

fig = px.pie(
    data_frame=df_sintomaticos_permaneceu_casa, 
    values = 'qtd_sintomaticos', 
    names = 'questao_permaneceu_casa',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={'
        'qtd_sintomaticos': 'Quantidade',
        'questao_permaneceu_casa': 'Isolamento'
    }
)

fig.update_layout(
    title='<b>Porcentagem de pacientes sintomáticos que adotaram medida de isolamento social</b>',
    legend_title='Legenda',
    width=800, 
    height=600
)

fig.show()

**Qual é a porcetagem de pacientes sintomáticos que efetuou medida de isolamento social por região?**

In [ ]:
df_sintomaticos_total_permaneceu_casa_regiao = spark.sql(
    '''
        SELECT regiao, count(sintoma_covid) as qtd_sintomaticos_total
        FROM df_temp 
        WHERE sintoma_covid = 'Sim'
        GROUP BY regiao
    '''
).toPandas()

df_sintomaticos_isolamento_permaneceu_casa_regiao = spark.sql(
    '''
        SELECT regiao, questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos_isolamento
        FROM df_temp 
        WHERE sintoma_covid = 'Sim' AND questao_permaneceu_casa = 'Sim'
        GROUP BY regiao, questao_permaneceu_casa
    '''
).toPandas()

df_permaneceu_casa_regiao = pd.merge(df_sintomaticos_total_permaneceu_casa_regiao, df_sintomaticos_isolamento_permaneceu_casa_regiao, on='regiao')
df_permaneceu_casa_regiao['percentual_sintomaticos_permaneceu_casa'] = ((df_permaneceu_casa_regiao.qtd_sintomaticos_isolamento / df_permaneceu_casa_regiao.qtd_sintomaticos_total) * 100).round(decimals=2)

fig = px.pie(
    data_frame=df_permaneceu_casa_regiao, 
    values = 'percentual_sintomaticos_permaneceu_casa', 
    names = 'regiao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_sintomaticos_permaneceu_casa': 'Percentual de pacientes sintomáticos que adotaram medida de isolamento social',
        'regiao': 'Região'
    }
)

fig.update_layout(
    title='<b>Porcentagem de pacientes sintomáticos que adotaram medida de isolamento social por região</b>',
    legend_title='Legenda',
    width=1000, 
    height=600
)

fig.show()

**Qual é a porcetagem de pacientes sintomáticos que efetuou medida de isolamento social por estado?**

In [ ]:
df_sintomaticos_total_permaneceu_casa_estado = spark.sql(
    '''
        SELECT uf, count(sintoma_covid) as qtd_sintomaticos_total
        FROM df_temp 
        WHERE sintoma_covid = 'Sim'
        GROUP BY uf
    '''
).toPandas()

df_sintomaticos_isolamento_permaneceu_casa_estado = spark.sql(
    '''
        SELECT uf, questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos_isolamento
        FROM df_temp 
        WHERE sintoma_covid = 'Sim' AND questao_permaneceu_casa = 'Sim'
        GROUP BY uf, questao_permaneceu_casa
    '''
).toPandas()

df_permaneceu_casa_estado = pd.merge(df_sintomaticos_total_permaneceu_casa_estado, df_sintomaticos_isolamento_permaneceu_casa_estado, on='uf')
df_permaneceu_casa_estado['percentual_sintomaticos_permaneceu_casa'] = ((df_permaneceu_casa_estado.qtd_sintomaticos_isolamento / df_permaneceu_casa_estado.qtd_sintomaticos_total) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_permaneceu_casa_estado.sort_values('percentual_sintomaticos_permaneceu_casa', ascending=False), 
    x='uf',
    y='percentual_sintomaticos_permaneceu_casa',
    color='uf',
    color_discrete_sequence=['#8B0000'],
    labels={
        'percentual_sintomaticos_permaneceu_casa': 'Percentual de pacientes sintomáticos que adotaram medida de isolamento social',
        'uf': 'Estado'
    }
)

fig.update_layout(
    title='<b>Porcentagem de pacientes sintomáticos que adotaram medida de isolamento social por estado</b>',
    yaxis_title='%',
    showlegend=False,
    width=1000, 
    height=600
)

fig.show()